In [ ]:
# conda install -c anaconda ipykernel
# %pip install ipywidgets --upgrade

# Install required libraries
# %pip install git+https://github.com/openai/whisper.git
# %pip install ipython
# %pip install torch
# %pip install pyannote.audio
# %pip install pydub
# %pip install webvtt-py
# %pip install yt-dlp
# %pip install lmstudio

In [1]:
# Download audio using yt-dlp
!yt-dlp -xv --audio-format wav -o "results/meeting_audio.wav" -- "https://www.youtube.com/watch?v=6dZBTtwF2-s"

[youtube] Extracting URL: https://www.youtube.com/watch?v=6dZBTtwF2-s
[youtube] 6dZBTtwF2-s: Downloading webpage
[youtube] 6dZBTtwF2-s: Downloading tv client config
[youtube] 6dZBTtwF2-s: Downloading player 73381ccc
[youtube] 6dZBTtwF2-s: Downloading tv player API JSON
[youtube] 6dZBTtwF2-s: Downloading ios player API JSON
[youtube] 6dZBTtwF2-s: Downloading m3u8 information
[info] 6dZBTtwF2-s: Downloading 1 format(s): 251
[download] results\meeting_audio.wav has already been downloaded
[ExtractAudio] Destination: results\meeting_audio.wav
Deleting original file results\meeting_audio.orig.wav (pass -k to keep)


[debug] Command-line config: ['-xv', '--audio-format', 'wav', '-o', 'results/meeting_audio.wav', '--', 'https://www.youtube.com/watch?v=6dZBTtwF2-s']
[debug] Encodings: locale utf-8, fs utf-8, pref utf-8, out utf-8 (No VT), error utf-8 (No VT), screen utf-8 (No VT)
[debug] yt-dlp version stable@2025.03.27 from yt-dlp/yt-dlp [48be862b3] (pip)
[debug] Python 3.11.11 (CPython AMD64 64bit) - Windows-10-10.0.26100-SP0 (OpenSSL 3.0.16 11 Feb 2025)
[debug] exe versions: ffmpeg 7.1-full_build-www.gyan.dev (setts), ffprobe 7.1-full_build-www.gyan.dev
[debug] Optional libraries: certifi-2025.01.31, requests-2.32.3, sqlite3-3.45.3, urllib3-2.3.0
[debug] Proxy map: {}
[debug] Request Handlers: urllib, requests
[debug] Plugin directories: none
[debug] Loaded 1849 extractors
[debug] Loading youtube-nsig.73381ccc from cache
[debug] [youtube] Decrypted nsig HOiqv6VTUumElDM_9 => E7ZFtUo9WkY3zQ
[debug] [youtube] Decrypted nsig noc-CbqqqolMFi3Cj => g8DP2p_T9X1A3g
[debug] [youtube] 6dZBTtwF2-s: ios client

In [1]:
from pydub import AudioSegment

# Trim the audio to the desired duration (for testing purposes)
# audio = AudioSegment.from_wav("results/meeting_audio.wav")
# trimmed_audio = audio[:20 * 60 * 1000]  # First 20 minutes
# trimmed_audio.export("results/trimmed_audio.wav", format="wav")

#Spacer for pyannote because pyannote misses first 0.5 seconds
audio = AudioSegment.from_wav("results/meeting_audio.wav")
spacermilli = 2000
spacer = AudioSegment.silent(duration=spacermilli)
audio = spacer.append(audio, crossfade=0)

audio.export("results/audio.wav", format="wav")

<_io.BufferedRandom name='results/audio.wav'>

In [ ]:
from pyannote.audio import Pipeline
from pyannote.core import Annotation
import os

auth_token = os.getenv("HF_AUTH_TOKEN")

# Load Pyannote's speaker diarization pipeline
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token=auth_token)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
d:\Projects\meeting_minutes_generator\.conda\Lib\inspect.py:988: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\laksh\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b\pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cpu. Bad things might happen unless you revert torch to 1.x.


d:\Projects\meeting_minutes_generator\.conda\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
d:\Projects\meeting_minutes_generator\.conda\Lib\site-packages\speechbrain\utils\autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
d:\Projects\meeting_minutes_generator\.conda\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `collect_in` in Pretrainer to avo

In [4]:
DEMO_FILE = {"uri": "meeting", "audio": "results/audio.wav"}
dz = pipeline(DEMO_FILE)

with open("results/diarization.txt", "w") as text_file:
    text_file.write(str(dz))
# print(*list(dz.itertracks(yield_label = True))[:10], sep="\n")

In [2]:
import re

# Function to convert time string to milliseconds
def millisec(timeStr):
    spl = timeStr.split(":")
    return int((int(spl[0]) * 3600 + int(spl[1]) * 60 + float(spl[2])) * 1000)

# Read diarization results
dz = open('results/diarization.txt').read().splitlines()

# Create a mapping for speaker labels to generic names
speaker_mapping = {}
speaker_counter = 0

# Process diarization results
dzList = []
for l in dz:
    # Extract start and end times
    start, end = tuple(re.findall(r'[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=l))
    start = millisec(start) - spacermilli
    end = millisec(end) - spacermilli

    # Extract speaker label
    speaker_label = re.search(r'SPEAKER_[0-9]+', l).group(0)

    # Map speaker label to a generic name if not already mapped
    if speaker_label not in speaker_mapping:
        speaker_mapping[speaker_label] = f"Speaker {speaker_counter}"
        speaker_counter += 1

    # Append the processed data
    dzList.append([start, end, speaker_mapping[speaker_label]])

# Print the mapping and processed diarization list
print("Speaker Mapping:", speaker_mapping)
print("Processed Diarization List:", dzList)

Speaker Mapping: {'SPEAKER_02': 'Speaker 0', 'SPEAKER_03': 'Speaker 1', 'SPEAKER_01': 'Speaker 2', 'SPEAKER_04': 'Speaker 3', 'SPEAKER_00': 'Speaker 4', 'SPEAKER_05': 'Speaker 5'}
Processed Diarization List: [[106, 17184, 'Speaker 0'], [15361, 16087, 'Speaker 1'], [18348, 26465, 'Speaker 0'], [18365, 18989, 'Speaker 1'], [28135, 66813, 'Speaker 0'], [67640, 102706, 'Speaker 0'], [103398, 104646, 'Speaker 0'], [105372, 132862, 'Speaker 0'], [134701, 168130, 'Speaker 0'], [169464, 172315, 'Speaker 0'], [173260, 185242, 'Speaker 0'], [187030, 204159, 'Speaker 0'], [204850, 212714, 'Speaker 0'], [213862, 232406, 'Speaker 0'], [234112, 273835, 'Speaker 1'], [274443, 280990, 'Speaker 1'], [281834, 283235, 'Speaker 1'], [284568, 285024, 'Speaker 1'], [285024, 285057, 'Speaker 2'], [286880, 290204, 'Speaker 2'], [290879, 292027, 'Speaker 2'], [293022, 293427, 'Speaker 2'], [294322, 295182, 'Speaker 2'], [295182, 295233, 'Speaker 1'], [301342, 301966, 'Speaker 2'], [304295, 315348, 'Speaker 2']

In [ ]:
#Segment the audio based on diarization results
from pydub import AudioSegment
import re

sounds = spacer
segments = []

dz = open('results/diarization.txt').read().splitlines()
for l in dz:
  start, end =  tuple(re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=l))
  start = int(millisec(start)) #milliseconds
  end = int(millisec(end))  #milliseconds

  segments.append(len(sounds))
  sounds = sounds.append(audio[start:end], crossfade=0)
  sounds = sounds.append(spacer, crossfade=0)

sounds.export("results/dz.wav", format="wav") #Exports to a wav file in the current path.

<_io.BufferedRandom name='results/dz.wav'>

In [ ]:
#Transcribe the audio using Whisper
!whisper results/dz.wav --language en --model turbo --output_format vtt --output_dir results

[00:00.000 --> 00:09.600]  We are co-chair of Pi Data Berlin and we have here also Theo and Valentina from the

d:\Projects\meeting_minutes_generator\.conda\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



[00:09.600 --> 00:16.000]  Pi Data Berlin team. As far as I can see, maybe some others.
[00:16.000 --> 00:18.000]  Daniel, I guess.
[00:18.000 --> 00:19.000]  Who?
[00:19.000 --> 00:22.000]  Daniel, I guess.
[00:22.000 --> 00:24.000]  Daniel?
[00:24.000 --> 00:29.000]  Daniel? Ah, hi Daniel. Yes, we are Pi Data Berlin.
[00:29.000 --> 00:36.000]  We are Daniel.
[00:36.000 --> 00:43.000]  Yes, we are the building chapter of Pi Data and Pi Data, as some of you may have heard before, is the
[00:43.000 --> 00:52.000]  educational program of NumFocus and we will talk about, it will be introduced very soon.
[00:52.000 --> 00:58.000]  Our program for today is me, I am your introduction, happening now.
[00:58.000 --> 01:11.000]  And then we have two very cool thoughts. One is from Katia Flores Poza, who will talk about controlling many satellites at the same time.
[01:11.000 --> 01:26.000]  And then we will have the second talk from Miladri Shekhardut about deepfake and why he has trust issues

In [ ]:
#Convert VTT to text
import webvtt

captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)),  caption.text] for caption in webvtt.read('results/dz.vtt')]
# print(*captions[:8], sep='\n')
# Extract text for each speaker
speaker_texts = {speaker: "" for speaker in speaker_mapping.values()}

for segment in dzList:
    start, end, speaker = segment
    for caption in captions:
        if caption[0] >= start and caption[1] <= end:
            speaker_texts[speaker] += " " + caption[2]

# Print the extracted text for each speaker
for speaker, text in speaker_texts.items():
    print(f"{speaker}: {text.strip()}")

Speaker 0: Pi Data Berlin team. As far as I can see, maybe some others. Daniel, I guess. Daniel? We are Daniel. Yes, we are the building chapter of Pi Data and Pi Data, as some of you may have heard before, is the educational program of NumFocus and we will talk about, it will be introduced very soon. Our program for today is me, I am your introduction, happening now. And then we will have the second talk from Miladri Shekhardut about deepfake and why he has trust issues. Let's hope we can solve them to some. Yeah, some of them today. In between, we will have a five minute break. And at the end, we can all meet up in this nice platform called Wonder. And later we will post in the chat the link so we can all gather there later. This meetup is recorded, so both talks are recorded and are also streamed now in YouTube, so please take it into consideration. Also, if you are watching on YouTube, you can also write down the questions in the YouTube chat and we will pass them on to the speaker

In [ ]:
# # Speaker Identification
# import re
# import json
# import lmstudio as lms

# # Initialize the model
# model = lms.llm("gemma-3-12b-it")

# # Structure the data into a JSON format
# structured_data = []
# for speaker, text in speaker_texts.items():
#     structured_data.append({
#         "speaker": speaker,
#         "text": text.strip()
#     })

# # Save the structured data to a JSON file (optional, for debugging purposes)
# with open("results/structured_data.json", "w") as json_file:
#     json.dump(structured_data, json_file, indent=4)

# # JSON schema 
# schema = {
#     "type": "array",
#     "items": {
#         "type": "object",
#         "properties": {
#             "speaker": { "type": "string" },
#             "entity": { "type": "string" },
#             "type": { "type": "string", "enum": ["PERSON"] }
#         },
#         "required": ["speaker", "entity", "type"]
#     }
# }

# # Function to perform NER using the lmstudio model
# def perform_ner_all(data):
#     # Define the refined prompt
#     prompt = (
#         f"Identify the real names of the different speaker labels from the following structured data and return them in JSON format. "
#         f"Focus on extracting PERSON entities for each speaker:\n\n"
#         f"{json.dumps(data, indent=4)}\n\n"
#     )
    
#     # Use the lmstudio model to generate a response
#     response = model.respond(prompt, response_format=schema)
    
#     # Check if the response is a string or an object
#     if isinstance(response, str):
#         return response  # If it's a string, return it directly
#     elif hasattr(response, "content"):
#         return response.content  # If it has a 'content' attribute, return it
#     else:
#         raise ValueError("Unexpected response format from model.respond()")

# # Perform NER for all speakers at once
# print("Processing NER for all speakers...")
# ner_response = perform_ner_all(structured_data)

# # Parse the response to map real names to speakers
# real_name_mapping = {}
# print("Raw NER Response:", ner_response)

# # Parse the JSON response from the model
# try:
#     ner_result = json.loads(ner_response.strip())
# except json.JSONDecodeError as e:
#     print(f"Error decoding JSON response: {e}")
#     ner_result = []


# # Map generic speaker labels to real names dynamically
# for entry in ner_result:
#     if entry.get("type") == "PERSON":
#         real_name_mapping[entry["entity"]] = entry["speaker"]

# # Print the real name mapping
# print("\nReal Name Mapping:")
# for speaker, real_name in real_name_mapping.items():
#     print(f"{speaker}: {real_name}")

# # Update dzList with real speaker names
# for segment in dzList:
#     generic_label = segment[2]  # The generic speaker label (e.g., "Speaker 0")
#     if generic_label in real_name_mapping:
#         segment[2] = real_name_mapping[generic_label]  # Replace with the real name

In [5]:
# Construct the transcript
transcript = ""
for i in range(len(segments)):
    idx = 0
    for idx in range(len(captions)):
        if captions[idx][0] >= (segments[i] - spacermilli):
            break

    while (idx < len(captions)) and ((i == len(segments) - 1) or (captions[idx][1] < segments[i + 1])):
        c = captions[idx]

        start = dzList[i][0] + (c[0] - segments[i])
        if start < 0:
            start = 0
        idx += 1

        start = start / 1000.0
        startStr = '{0:02d}:{1:02d}:{2:02.2f}'.format(
            int(start // 3600),
            int(start % 3600 // 60),
            start % 60
        )

        # Include speaker name and text in the transcript
        speaker_name = dzList[i][2]
        transcript += f"[{startStr}] {speaker_name}: {c[2]}\n"

# Save the transcript to a file
with open("results/transcript.txt", "w", encoding="utf-8") as file:
    file.write(transcript)

In [6]:
# Construct the transcript without timestamps
transcript_no_timestamps = ""
for i in range(len(segments)):
    idx = 0
    for idx in range(len(captions)):
        if captions[idx][0] >= (segments[i] - spacermilli):
            break

    while (idx < len(captions)) and ((i == len(segments) - 1) or (captions[idx][1] < segments[i + 1])):
        c = captions[idx]
        idx += 1

        # Include only speaker name and text in the transcript
        speaker_name = dzList[i][2]
        transcript_no_timestamps += f"{speaker_name}: {c[2]}\n"

# Save the transcript without timestamps to a file
with open("results/transcript_no_timestamps.txt", "w", encoding="utf-8") as file:
    file.write(transcript_no_timestamps)

In [ ]:
# import json
# import lmstudio as lms

# # Initialize the model
# model = lms.llm("gemma-3-12b-it")

# # Define the prompt for summarization
# prompt = (
#     "Please summarize the following meeting transcript into meeting minutes with HTML formatting using various tags to fit inside a <div>. "
#     "This is an API call, do not give any explanation,affirmation,etc.\n\n"
#     "Focus on agenda,key points, decisions, and action items:\n\n"
#     f"{transcript}"
# )

# # Function to summarize the transcript using the lmstudio model
# def summarize_transcript(prompt):
#     # Use the lmstudio model to generate a response
#     response = model.respond(prompt)
    
#     # Check if the response is a string or an object
#     if isinstance(response, str):
#         return response.strip()  # If it's a string, return it directly
#     elif hasattr(response, "content"):
#         return response.content.strip()  # If it has a 'content' attribute, return it
#     else:
#         raise ValueError("Unexpected response format from model.respond()")

# # Generate the summary
# print("Generating summarized meeting minutes...")
# summary = summarize_transcript(prompt)

# # Print the summarized meeting minutes
# print("\nSummarized Meeting Minutes:\n", summary)

# # Save the summary to a file
# with open("results/meeting_minutes.txt", "w", encoding="utf-8") as file:
#     file.write(summary)

In [9]:
#Summarize the transcript into meeting minutes with an LLM
import json
import lmstudio as lms

# Load the JSON transcript
with open("results/transcript_no_timestamps.json", "r", encoding="utf-8") as file:
    transcript_data = json.load(file)

# Convert JSON data to a plain text string
transcript_text = json.dumps(transcript_data)

# Initialize the model
model = lms.llm("gemma-3-12b-it")

# Function to call the model with a prompt
def summarize_text(text, is_final=False):
    prompt_type = (
        "Here are partial meeting transcripts. Summarize the agenda, key points, decisions, and action items.This is an API call, do not give any explanation,affirmation,etc.\n\n"
        if not is_final else
        "Please combine and summarize the following chunk summaries into polished meeting minutes using  required HTML tags inside a <div>.Focus on agenda,key points, decisions, and action items.This is an API call, do not give any explanation,affirmation,etc.\n\n"
    )
    prompt = prompt_type + text
    response = model.respond(prompt)
    if isinstance(response, str):
        return response.strip()
    elif hasattr(response, "content"):
        return response.content.strip()
    else:
        raise ValueError("Unexpected response format from model.respond()")

# Split text into chunks
def chunk_text(text, chunk_size=8000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    return chunks

# Chunk the transcript
chunks = chunk_text(transcript_text)

# Summarize each chunk
chunk_summaries = []
print("Summarizing chunks...")
for i, chunk in enumerate(chunks):
    print(f"Summarizing chunk {i+1}/{len(chunks)}...")
    summary = summarize_text(chunk)
    chunk_summaries.append(summary)

# Combine all chunk summaries into one final summary
combined_summary = "\n\n".join(chunk_summaries)
print("Generating final meeting minutes...")
final_summary = summarize_text(combined_summary, is_final=True)

# Remove the ```html at the start and ``` at the end
if final_summary.startswith("```html"):
    final_summary = final_summary[7:]  # Remove the first 7 characters (```html)
if final_summary.endswith("```"):
    final_summary = final_summary[:-3]  # Remove the last 3 characters (```)

# Output and save
print("\nFinal Meeting Minutes:\n", final_summary)

with open("results/meeting_minutes.txt", "w", encoding="utf-8") as file:
    file.write(final_summary)


Summarizing chunks...
Summarizing chunk 1/11...
Summarizing chunk 2/11...
Summarizing chunk 3/11...
Summarizing chunk 4/11...
Summarizing chunk 5/11...
Summarizing chunk 6/11...
Summarizing chunk 7/11...
Summarizing chunk 8/11...
Summarizing chunk 9/11...
Summarizing chunk 10/11...
Summarizing chunk 11/11...
Generating final meeting minutes...

Final Meeting Minutes:
 
<div>
  <h2>Pi Data Berlin Meetup - Meeting Minutes</h2>

  <h3>Date: [Insert Date]</h3>

  <h4>Agenda</h4>
  <ul>
    <li>Introduction to Pi Data Berlin</li>
    <li>Space Operations Talks (Katia Flores Poza)</li>
    <li>Deepfake Technology and Detection Discussions</li>
  </ul>

  <h4>Key Points & Summary</h4>

  <p>This meeting, hosted by the local chapter of Pi Data Berlin (an educational program by NumFocus focused on Python data science), featured presentations on space operations and deepfake technology. The meetups occur monthly on the third Wednesday.</p>

  <h5>Space Operations</h5>
  <ul>
    <li><b>Introduct

In [10]:
# Create HTML file with meeting minutes and transcript
preS = '''<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Meeting Minutes and Transcript</title>
    <style>
        body {
            font-family: sans-serif;
            font-size: 18px;
            color: #111;
            padding: 0 0 1em 0;
        }
        .l {
            color: #050;
        }
        .s, .e, .t {
            display: inline-block;
        }
        #player {
            position: sticky;
            top: 20px;
            float: right;
        }
    </style>
  </head>
  <body>
    <h2>Meeting Minutes and Transcript</h2>
    <div id="player"></div>
    <script>
      var tag = document.createElement('script');
      tag.src = "https://www.youtube.com/iframe_api";
      var firstScriptTag = document.getElementsByTagName('script')[0];
      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);

      var player;
      function onYouTubeIframeAPIReady() {
        player = new YT.Player('player', {
          height: '210',
          width: '340',
          videoId: '6dZBTtwF2-s',
        });
      }

      function setCurrentTime(timepoint) {
        player.seekTo(timepoint);
        player.playVideo();
      }
    </script>
    <br>
'''
postS = '''  </body>
</html>'''

html = list(preS)
html.append('<div class="minutes">\n')
html.append(f'{final_summary}\n')  # Add the HTML-formatted meeting minutes directly
html.append('</div>\n')
html.append('<hr>\n')
html.append('<h2>Meeting Transcript</h2>\n')

# Generate the transcript section
for line in transcript.splitlines():
    # Extract timestamp, speaker name, and text from the transcript line
    match = re.match(r'\[(.*?)\] (.*?): (.*)', line)
    if match:
        timestamp, speaker_name, text = match.groups()
        html.append('\t\t\t<div class="c">\n')
        html.append(f'\t\t\t\t<a class="l" href="#{timestamp}" id="{timestamp}">link</a> |\n')
        html.append(f'\t\t\t\t<div class="s"><a href="javascript:void(0);" onclick=setCurrentTime({int(float(timestamp.split(":")[0]) * 3600 + float(timestamp.split(":")[1]) * 60 + float(timestamp.split(":")[2]))})>{timestamp}</a></div>\n')
        html.append(f'\t\t\t\t<div class="t"><strong>{speaker_name}:</strong> {text}</div>\n')
        html.append('\t\t\t</div>\n\n')


html.append(postS)

# Write the HTML content to the file
s = "".join(html)
with open("results/meeting.html", "w") as text_file:
    text_file.write(s)

print(s)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Meeting Minutes and Transcript</title>
    <style>
        body {
            font-family: sans-serif;
            font-size: 18px;
            color: #111;
            padding: 0 0 1em 0;
        }
        .l {
            color: #050;
        }
        .s, .e, .t {
            display: inline-block;
        }
        #player {
            position: sticky;
            top: 20px;
            float: right;
        }
    </style>
  </head>
  <body>
    <h2>Meeting Minutes and Transcript</h2>
    <div id="player"></div>
    <script>
      var tag = document.createElement('script');
      tag.src = "https://www.youtube.com/iframe_api";
      var firstScriptTag = document.getElementsByTagName('script')[0];
      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);

      v